## Objectif
Ce notebook vise à explorer les techniques d'augmentation de données d'images et à construire un modèle de classification pour identifier les pneumonies à partir d'images de rayons X. Les étapes clés incluent l'augmentation des données, l'entraînement d'un modèle CNN et l'évaluation des performances du modèle.

## Étapes de Démarche

1. **Vérification de la Disponibilité du GPU**
2. **Chargement et Préparation des Données**
   - Chargement, redimensionnement, conversion en niveaux de gris, normalisation et restructuration des images.
3. **Visualisation des Données**
4. **Augmentation des Données**
5. **Définition et Entraînement du Modèle CNN**
6. **Évaluation du Modèle**
7. **Visualisation des Performances**
   - Matrice de confusion, rapport de classification et courbe ROC.
8. **Heatmaps des Activations**

## Interprétation et Résultats
- **Courbes d'Apprentissage** : convergence du modèle et relation entre la perte et l'exactitude.
- **Matrice de Confusion** : répartition des erreurs de classification.
- **Rapport de Classification** : mesures détaillées comme la précision, le rappel et le score F1.
- **Courbe ROC** : capacité du modèle à distinguer les classes à différents seuils.
- **Heatmaps des Activations** : caractéristiques extraites par les couches du modèle.ion des données d'entrée.

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_curve, auc
import seaborn as sns

Vérifie si des GPUs sont disponibles parce que si ils le sont alors ça accélère les calculs lors de l'entraînement du modèle, donc ça réduit le temps d'entaînement

In [7]:
print("Nombre de GPUs disponibles : ", len(tf.config.experimental.list_physical_devices('GPU')))

Nombre de GPUs disponibles :  0


Définition des constantes

In [12]:
img_size = 150 #150x150 pixels
data_dir1 = '../../../chest_xray/train/PNEUMONIA'
data_dir2 = '../../../chest_xray/train/NORMAL'

Charger et prétraiter les images à partir des 2 répertoires spécifiés. Cette fonction prend en entrée deux chemins de répertoire (data_dir1 et data_dir2) et retourne un tableau contenant les images redimensionnées et leurs étiquettes respectives.

In [ ]:
def load_data(data_dir1, data_dir2):
    data = []
    labels = ['PNEUMONIA', 'NORMAL']
    for dir in [data_dir1, data_dir2]:
        path = os.path.join(dir)
        if not os.path.exists(path):
            print(f"Le répertoire {path} n'existe pas.")
            continue
        label = os.path.basename(dir)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                if img_arr is None:
                    print(f"Échec de lecture de {img}. Ignoré.")
                    continue
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data, dtype=object)

train_data = load_data(data_dir1, data_dir2)

Visualisation du nombre d'images par classe

In [ ]:
labels_count = [0, 0]
for item in train_data:
    labels_count[item[1]] += 1

plt.bar(['PNEUMONIA', 'NORMAL'], labels_count)
plt.title('Nombre d\'images par classe')
plt.show()

Prendre en entrée les données (features et labels) et les sépare en deux tableaux distincts : un pour les features (images) et un autre pour les labels (étiquettes de classe).

- **Normaliser** : 
  - Chaque pixel d'une image a une valeur entre 0 et 255.
  - Normaliser signifie diviser ces valeurs par 255 pour qu'elles soient entre 0 et 1.
 Caela facilite l'entraînement du modèparce quecar les valeurs sont plus petites et plus uniformes.

- **Reshaper** :
  - Les images sont des tableaux de nombres (valeurs des pixels).
  - Reveut direignifie réorganiser ces tableaux pour qu'ils aient une forme spécifique.
  - On veut que chaque image ait la forme (nombre d'images, 150, 150, 1).
  - Cela indique qu'on a un certain nombre d'images, que chaque image a une taille de 150x150 pixels, et qu'elle est en niveaux de gris (1 canal de couleur).


In [2]:
def separate_features_labels(data):
    features, labels = [], []
    # séparation des features et des labels
    for feature, label in data:
        features.append(feature)
        labels.append(label)
    return np.array(features), np.array(labels)

#normaliser les données en divisant par 255
x_train, y_train = separate_features_labels(train_data)

# Normaliser les données
x_train = x_train / 255.0
x_train = x_train.reshape(-1, img_size, img_size, 1)

NameError: name 'train_data' is not defined